# Week 2: Quantitative Measures of Population Health — DALYs and QALYs

**Learning Objectives:**
- Understand the components of Disability-Adjusted Life Years (DALYs)
- Calculate Years of Life Lost (YLL) and Years Lived with Disability (YLD)
- Explore how disability weights are determined and their impact on burden estimates
- Compare DALYs and Quality-Adjusted Life Years (QALYs)
- Critically evaluate the assumptions embedded in these measures

---

## 1. Why Do We Need Summary Measures of Health?

Traditional health statistics focus on **mortality** — death rates, life expectancy, cause of death. But this misses something important: **morbidity**.

Consider two conditions:
- **Ischaemic heart disease**: High mortality, significant disability before death
- **Low back pain**: Very low mortality, but affects millions and causes significant suffering

If we only count deaths, we systematically undervalue conditions that cause suffering without killing. Summary measures like DALYs and QALYs attempt to capture **both** mortality and morbidity in a single metric.

### The Policy Question

Imagine you're advising a health ministry with a fixed budget. How do you compare:
- Preventing 100 deaths from heart disease?
- Preventing 10,000 cases of chronic low back pain?
- Curing 500 cases of blindness?

To make these comparisons, we need a **common currency** for health.

## 2. Setup and Data

Let's start by loading the libraries we'll need and some example data.

In [ ]:
# Install required packages (run once)
!pip install pandas numpy matplotlib ipywidgets plotly -q

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from ipywidgets import interact, FloatSlider, IntSlider, VBox, HBox, Label, Output
import ipywidgets as widgets
from IPython.display import display, Markdown, HTML

# Set plotting style
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = [10, 6]

In [ ]:
# GBD 2019 Reference Life Table (abridged)
# This represents the "ideal" life expectancy at each age
life_table = pd.DataFrame({
    'age': [0, 1, 5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70, 75, 80, 85, 90, 95],
    'life_expectancy': [88.9, 88.0, 84.0, 79.0, 74.1, 69.1, 64.2, 59.2, 54.3, 49.4, 44.5, 
                        39.7, 35.0, 30.4, 25.9, 21.6, 17.5, 13.7, 10.3, 7.4, 5.0]
})

print("GBD 2019 Reference Life Table")
print("="*40)
display(life_table)

In [ ]:
# GBD Disability Weights for selected conditions
# These are from GBD 2019 - values range from 0 (perfect health) to 1 (equivalent to death)

gbd_disability_weights = pd.DataFrame({
    'condition': [
        'Mild anaemia',
        'Moderate hearing loss',
        'Moderate depression',
        'Severe low back pain',
        'Blindness',
        'Moderate heart failure',
        'Severe dementia',
        'Untreated spinal cord injury (below neck)',
        'Terminal cancer with severe pain',
        'Type 2 diabetes without complications',
        'Type 2 diabetes with diabetic foot',
        'Obesity (class III, BMI >= 40)',
        'Iron deficiency'
    ],
    'disability_weight': [
        0.004,
        0.027,
        0.145,
        0.325,
        0.187,
        0.072,
        0.449,
        0.589,
        0.569,
        0.015,
        0.133,
        0.086,
        0.004
    ],
    'category': [
        'Nutritional deficiency',
        'Sensory',
        'Mental health',
        'Musculoskeletal',
        'Sensory',
        'Cardiovascular',
        'Neurological',
        'Injury',
        'Cancer',
        'Metabolic',
        'Metabolic',
        'Metabolic',
        'Nutritional deficiency'
    ]
})

print("GBD 2019 Disability Weights (selected conditions)")
print("="*60)
display(gbd_disability_weights.sort_values('disability_weight', ascending=False))

## 3. Building DALYs from First Principles

The DALY combines two components:

$$\text{DALY} = \text{YLL} + \text{YLD}$$

Where:
- **YLL** (Years of Life Lost) = years lost due to premature mortality
- **YLD** (Years Lived with Disability) = years lived in less than perfect health

**Key insight**: 1 DALY = 1 year of healthy life lost. Higher DALYs = greater burden.

### 3.1 Years of Life Lost (YLL)

YLL measures premature mortality by comparing actual age at death to a reference life expectancy:

$$\text{YLL} = N \times L$$

Where:
- $N$ = number of deaths
- $L$ = standard life expectancy at age of death

In [ ]:
def get_life_expectancy(age, life_table=life_table):
    """
    Get remaining life expectancy at a given age using linear interpolation.
    """
    return np.interp(age, life_table['age'], life_table['life_expectancy'])

def calculate_yll(deaths_by_age):
    """
    Calculate Years of Life Lost from a dictionary of {age: number_of_deaths}.
    
    Parameters:
    -----------
    deaths_by_age : dict
        Dictionary mapping age at death to number of deaths
    
    Returns:
    --------
    float : Total YLL
    pd.DataFrame : Breakdown by age
    """
    results = []
    total_yll = 0
    
    for age, n_deaths in deaths_by_age.items():
        le = get_life_expectancy(age)
        yll = n_deaths * le
        total_yll += yll
        results.append({
            'age_at_death': age,
            'n_deaths': n_deaths,
            'life_expectancy': round(le, 1),
            'yll': round(yll, 1)
        })
    
    return total_yll, pd.DataFrame(results)

In [ ]:
# Example: Mortality from ischaemic heart disease
# Simplified data - deaths at different ages

ihd_deaths = {
    45: 50,
    55: 200,
    65: 500,
    75: 800,
    85: 400
}

total_yll, yll_breakdown = calculate_yll(ihd_deaths)

print(f"Ischaemic Heart Disease - YLL Calculation")
print("="*50)
display(yll_breakdown)
print(f"\nTotal Deaths: {sum(ihd_deaths.values()):,}")
print(f"Total YLL: {total_yll:,.0f} years")
print(f"Average YLL per death: {total_yll/sum(ihd_deaths.values()):.1f} years")

### Exercise 3.1: Calculate YLL for a different condition

Road traffic accidents tend to affect younger people. Calculate the YLL using the mortality data below and compare to IHD.

In [ ]:
# Road traffic accident deaths (hypothetical data for a population)
rta_deaths = {
    15: 30,
    25: 80,
    35: 60,
    45: 40,
    55: 20,
    65: 10
}

# YOUR CODE HERE
# Calculate YLL for road traffic accidents
# Compare total deaths and total YLL between IHD and RTA



### 3.2 Years Lived with Disability (YLD)

YLD captures the morbidity burden:

$$\text{YLD} = P \times DW$$

Where:
- $P$ = prevalence (number of cases) × average duration
- $DW$ = disability weight (0 to 1)

Or equivalently:

$$\text{YLD} = I \times DW \times D$$

Where:
- $I$ = incidence (new cases)
- $D$ = average duration of condition

In [ ]:
def calculate_yld(prevalence, disability_weight, duration=1):
    """
    Calculate Years Lived with Disability.
    
    Parameters:
    -----------
    prevalence : int
        Number of prevalent cases (or incidence if using duration)
    disability_weight : float
        Disability weight (0-1)
    duration : float
        Average duration in years (default=1 for prevalence-based)
    
    Returns:
    --------
    float : YLD
    """
    return prevalence * disability_weight * duration

In [ ]:
# Example: Comparing YLD for different conditions

conditions = [
    {'name': 'Type 2 diabetes (uncomplicated)', 'prevalence': 100000, 'dw': 0.015},
    {'name': 'Moderate depression', 'prevalence': 20000, 'dw': 0.145},
    {'name': 'Severe low back pain', 'prevalence': 15000, 'dw': 0.325},
    {'name': 'Blindness', 'prevalence': 5000, 'dw': 0.187},
]

print("YLD Comparison (prevalence-based, 1 year)")
print("="*70)

yld_results = []
for c in conditions:
    yld = calculate_yld(c['prevalence'], c['dw'])
    yld_results.append({
        'Condition': c['name'],
        'Prevalence': f"{c['prevalence']:,}",
        'Disability Weight': c['dw'],
        'YLD': round(yld, 0)
    })

yld_df = pd.DataFrame(yld_results)
display(yld_df)

### 3.3 Putting it Together: Total DALYs

Now let's calculate total DALYs for a condition that has both mortality and morbidity.

In [ ]:
def calculate_dalys(deaths_by_age, prevalence, disability_weight, condition_name="Condition"):
    """
    Calculate total DALYs for a condition.
    """
    yll, _ = calculate_yll(deaths_by_age)
    yld = calculate_yld(prevalence, disability_weight)
    dalys = yll + yld
    
    print(f"\n{condition_name} - DALY Calculation")
    print("="*50)
    print(f"YLL (Years of Life Lost):        {yll:>12,.0f}")
    print(f"YLD (Years Lived with Disability): {yld:>10,.0f}")
    print(f"-"*50)
    print(f"Total DALYs:                     {dalys:>12,.0f}")
    print(f"\nYLL proportion: {yll/dalys*100:.1f}%")
    print(f"YLD proportion: {yld/dalys*100:.1f}%")
    
    return {'yll': yll, 'yld': yld, 'dalys': dalys}

In [ ]:
# Ischaemic Heart Disease: high mortality, moderate disability
ihd = calculate_dalys(
    deaths_by_age={45: 50, 55: 200, 65: 500, 75: 800, 85: 400},
    prevalence=50000,  # people living with IHD
    disability_weight=0.072,  # moderate heart failure
    condition_name="Ischaemic Heart Disease"
)

In [ ]:
# Low Back Pain: very low mortality, high disability burden
lbp = calculate_dalys(
    deaths_by_age={75: 5, 85: 10},  # minimal direct mortality
    prevalence=200000,  # very common
    disability_weight=0.325,  # severe pain
    condition_name="Severe Low Back Pain"
)

## 4. The Disability Weight Exercise

### 4.1 How GBD Determines Disability Weights

The Global Burden of Disease study uses **paired comparisons** from population surveys. Respondents are presented with two hypothetical health states and asked which they consider healthier.

For example:
> "Which person do you consider to be healthier overall?"
> - Person A has moderate difficulty walking and chronic pain
> - Person B has complete blindness but no pain

Thousands of these comparisons are analysed to derive a consistent scale from 0 (perfect health) to 1 (equivalent to death).

### 4.2 Your Turn: Set Your Own Disability Weights

In this exercise, you'll assign your own disability weights to a set of conditions. Later, we'll compare your weights to the GBD values and see how this affects disease burden rankings.

In [ ]:
# Conditions for the disability weight exercise
exercise_conditions = [
    {
        'name': 'Mild anaemia',
        'description': 'Feels slightly tired and weak at times. Some difficulty with physical activities.',
        'gbd_weight': 0.004
    },
    {
        'name': 'Moderate hearing loss',
        'description': 'Has difficulty hearing conversations in noisy environments. May need to ask people to repeat themselves.',
        'gbd_weight': 0.027
    },
    {
        'name': 'Moderate depression',
        'description': 'Feels sad and has lost interest in usual activities. Has difficulty sleeping and concentrating.',
        'gbd_weight': 0.145
    },
    {
        'name': 'Severe low back pain',
        'description': 'Has severe, constant pain in the lower back. Cannot do most daily activities and has difficulty sleeping.',
        'gbd_weight': 0.325
    },
    {
        'name': 'Complete blindness',
        'description': 'Cannot see at all. Needs assistance with many daily activities.',
        'gbd_weight': 0.187
    },
    {
        'name': 'Severe dementia',
        'description': 'Cannot remember recent events or recognise close family members. Needs constant supervision.',
        'gbd_weight': 0.449
    },
    {
        'name': 'Type 2 diabetes (controlled)',
        'description': 'Must monitor diet and take daily medication. No major symptoms if well-managed.',
        'gbd_weight': 0.015
    },
    {
        'name': 'Obesity (BMI ≥ 40)',
        'description': 'Has significant excess weight causing difficulty with physical activities and daily tasks.',
        'gbd_weight': 0.086
    }
]

In [ ]:
# Display conditions for students to consider
print("DISABILITY WEIGHT EXERCISE")
print("="*80)
print("\nAssign a disability weight (0-1) to each condition below.")
print("0 = perfect health, 1 = equivalent to death")
print("\nConsider: How much does this condition reduce quality of life?\n")
print("-"*80)

for i, c in enumerate(exercise_conditions, 1):
    print(f"\n{i}. {c['name'].upper()}")
    print(f"   {c['description']}")

In [ ]:
# Interactive widget for setting disability weights

# Create sliders for each condition
sliders = {}
slider_widgets = []

for c in exercise_conditions:
    slider = FloatSlider(
        value=0.1,
        min=0,
        max=1,
        step=0.01,
        description='',
        continuous_update=False,
        readout=True,
        readout_format='.2f',
        style={'description_width': 'initial'},
        layout=widgets.Layout(width='300px')
    )
    sliders[c['name']] = slider
    
    label = widgets.HTML(
        value=f"<b>{c['name']}</b>",
        layout=widgets.Layout(width='200px')
    )
    
    slider_widgets.append(HBox([label, slider]))

# Output area for comparison
output = Output()

def compare_weights(change=None):
    with output:
        output.clear_output(wait=True)
        
        comparison_data = []
        for c in exercise_conditions:
            your_weight = sliders[c['name']].value
            gbd_weight = c['gbd_weight']
            diff = your_weight - gbd_weight
            comparison_data.append({
                'Condition': c['name'],
                'Your Weight': your_weight,
                'GBD Weight': gbd_weight,
                'Difference': diff
            })
        
        df = pd.DataFrame(comparison_data)
        
        # Create comparison plot
        fig, ax = plt.subplots(figsize=(12, 6))
        
        x = np.arange(len(df))
        width = 0.35
        
        bars1 = ax.bar(x - width/2, df['Your Weight'], width, label='Your weights', color='steelblue')
        bars2 = ax.bar(x + width/2, df['GBD Weight'], width, label='GBD weights', color='coral')
        
        ax.set_ylabel('Disability Weight')
        ax.set_title('Your Disability Weights vs GBD 2019')
        ax.set_xticks(x)
        ax.set_xticklabels(df['Condition'], rotation=45, ha='right')
        ax.legend()
        ax.set_ylim(0, 1)
        
        plt.tight_layout()
        plt.show()
        
        # Show table
        print("\nComparison Table:")
        display(df.style.format({
            'Your Weight': '{:.3f}',
            'GBD Weight': '{:.3f}',
            'Difference': '{:+.3f}'
        }).background_gradient(subset=['Difference'], cmap='RdYlGn_r', vmin=-0.3, vmax=0.3))

# Add observer to each slider
for slider in sliders.values():
    slider.observe(compare_weights, names='value')

# Create compare button
compare_button = widgets.Button(description="Compare to GBD", button_style='primary')
compare_button.on_click(compare_weights)

print("Adjust the sliders to set your disability weights, then click 'Compare to GBD':")
print("")
display(VBox(slider_widgets + [compare_button, output]))

### 4.3 Discussion Questions

After comparing your weights to the GBD values:

1. **Where did you differ most from GBD?** Why might your valuation differ from population surveys?

2. **Adaptation**: People who have lived with a condition often rate it as less severe than those imagining it. Should disability weights reflect the valuations of patients or the general population?

3. **Cultural variation**: GBD uses surveys from multiple countries. Might blindness be valued differently in cultures with strong oral traditions? Might depression be valued differently depending on cultural attitudes to mental health?

4. **The "worse than death" problem**: GBD caps weights at 1.0, but some respondents rate certain health states as worse than death. How should we handle this?

## 5. How Disability Weights Change Rankings

Now let's see how your disability weights would change the burden rankings for a hypothetical population.

In [ ]:
# Hypothetical prevalence data for our population
population_data = {
    'Mild anaemia': {'prevalence': 500000, 'deaths_by_age': {}},
    'Moderate hearing loss': {'prevalence': 300000, 'deaths_by_age': {}},
    'Moderate depression': {'prevalence': 150000, 'deaths_by_age': {45: 50, 55: 30, 65: 20}},
    'Severe low back pain': {'prevalence': 100000, 'deaths_by_age': {}},
    'Complete blindness': {'prevalence': 30000, 'deaths_by_age': {}},
    'Severe dementia': {'prevalence': 80000, 'deaths_by_age': {75: 2000, 85: 5000}},
    'Type 2 diabetes (controlled)': {'prevalence': 400000, 'deaths_by_age': {55: 100, 65: 500, 75: 1000, 85: 500}},
    'Obesity (BMI ≥ 40)': {'prevalence': 200000, 'deaths_by_age': {45: 50, 55: 200, 65: 300, 75: 200}}
}

In [ ]:
def calculate_burden_rankings(population_data, disability_weights, weight_source="Custom"):
    """
    Calculate disease burden rankings using given disability weights.
    """
    results = []
    
    for condition, data in population_data.items():
        # Get disability weight
        dw = disability_weights.get(condition, 0.1)
        
        # Calculate YLL
        yll = 0
        if data['deaths_by_age']:
            yll, _ = calculate_yll(data['deaths_by_age'])
        
        # Calculate YLD
        yld = calculate_yld(data['prevalence'], dw)
        
        # Total DALYs
        dalys = yll + yld
        
        results.append({
            'Condition': condition,
            'Prevalence': data['prevalence'],
            'DW': dw,
            'YLL': yll,
            'YLD': yld,
            'DALYs': dalys
        })
    
    df = pd.DataFrame(results).sort_values('DALYs', ascending=False)
    df['Rank'] = range(1, len(df) + 1)
    
    return df[['Rank', 'Condition', 'Prevalence', 'DW', 'YLL', 'YLD', 'DALYs']]

In [ ]:
# Calculate rankings with GBD weights
gbd_weights = {c['name']: c['gbd_weight'] for c in exercise_conditions}
gbd_rankings = calculate_burden_rankings(population_data, gbd_weights, "GBD")

print("Disease Burden Rankings using GBD Disability Weights")
print("="*80)
display(gbd_rankings.style.format({
    'Prevalence': '{:,.0f}',
    'DW': '{:.3f}',
    'YLL': '{:,.0f}',
    'YLD': '{:,.0f}',
    'DALYs': '{:,.0f}'
}))

In [ ]:
# Calculate rankings with YOUR weights
your_weights = {name: slider.value for name, slider in sliders.items()}
your_rankings = calculate_burden_rankings(population_data, your_weights, "Your weights")

print("Disease Burden Rankings using YOUR Disability Weights")
print("="*80)
display(your_rankings.style.format({
    'Prevalence': '{:,.0f}',
    'DW': '{:.3f}',
    'YLL': '{:,.0f}',
    'YLD': '{:,.0f}',
    'DALYs': '{:,.0f}'
}))

In [ ]:
# Compare rankings side by side
comparison = pd.merge(
    gbd_rankings[['Condition', 'Rank', 'DALYs']].rename(columns={'Rank': 'GBD Rank', 'DALYs': 'GBD DALYs'}),
    your_rankings[['Condition', 'Rank', 'DALYs']].rename(columns={'Rank': 'Your Rank', 'DALYs': 'Your DALYs'}),
    on='Condition'
)
comparison['Rank Change'] = comparison['GBD Rank'] - comparison['Your Rank']
comparison = comparison.sort_values('GBD Rank')

print("\nRanking Comparison: GBD vs Your Weights")
print("="*80)
print("(Positive rank change = moved UP in your ranking)\n")

display(comparison.style.format({
    'GBD DALYs': '{:,.0f}',
    'Your DALYs': '{:,.0f}',
    'Rank Change': '{:+d}'
}).background_gradient(subset=['Rank Change'], cmap='RdYlGn', vmin=-3, vmax=3))

## 6. DALYs vs QALYs

While DALYs measure **burden** (higher = worse), QALYs measure **health gained** (higher = better). They're conceptually similar but used differently:

| Aspect | DALY | QALY |
|--------|------|------|
| **Direction** | Higher = more burden | Higher = more health |
| **Primary use** | Population burden estimation | Cost-effectiveness analysis |
| **Weights** | Disability weights (surveys) | Utility weights (patient preferences) |
| **Reference** | Ideal life expectancy | Actual remaining life expectancy |
| **Key user** | GBD, WHO | NICE, health economists |

### Converting Between Them

Roughly:
- 1 DALY averted ≈ 1 QALY gained
- QALY weight = 1 - disability weight (approximately)

In [ ]:
def calculate_qalys_gained(intervention_effect, population, duration, dw_before, dw_after):
    """
    Calculate QALYs gained from an intervention.
    
    Parameters:
    -----------
    intervention_effect : float
        Proportion of population that benefits (0-1)
    population : int
        Size of target population
    duration : float
        Years of benefit
    dw_before : float
        Disability weight before intervention
    dw_after : float
        Disability weight after intervention
    
    Returns:
    --------
    float : QALYs gained
    """
    utility_before = 1 - dw_before
    utility_after = 1 - dw_after
    utility_gain = utility_after - utility_before
    
    people_benefiting = population * intervention_effect
    qalys = people_benefiting * utility_gain * duration
    
    return qalys, utility_gain

In [ ]:
# Example: Cataract surgery restoring sight
qalys, utility_gain = calculate_qalys_gained(
    intervention_effect=0.95,  # 95% success rate
    population=10000,          # people needing surgery
    duration=15,               # years of benefit
    dw_before=0.187,           # blindness
    dw_after=0.003             # corrected vision
)

print("Cataract Surgery Programme")
print("="*50)
print(f"Population: 10,000 people with cataracts")
print(f"Success rate: 95%")
print(f"Duration of benefit: 15 years")
print(f"\nUtility gain per person: {utility_gain:.3f}")
print(f"Total QALYs gained: {qalys:,.0f}")
print(f"\nCost-effectiveness:")
print(f"  If programme costs £10M: £{10000000/qalys:,.0f} per QALY")
print(f"  NICE threshold: £20,000-£30,000 per QALY")

## 7. Limitations and Critique

### Embedded Value Judgements

DALYs and QALYs appear objective but encode significant value choices:

1. **Whose preferences count?** General population vs. patients vs. caregivers
2. **Is a year of life worth the same at all ages?** (Age-weighting, now deprecated by GBD)
3. **Should we discount future health?** (GBD no longer discounts; economists typically use 3-3.5%)
4. **Do these measures disadvantage disabled people?** A life-saving intervention for someone with a pre-existing disability generates fewer QALYs

### The Disability Rights Critique

Some disability advocates argue that QALYs and DALYs:
- Assume disabled lives are worth less
- Fail to distinguish between the condition and social barriers
- May lead to discrimination in resource allocation

In [ ]:
# Illustration: The disability discrimination problem

print("The QALY Discrimination Problem")
print("="*60)
print("\nScenario: A life-saving treatment that adds 10 years of life.")
print("Compare two patients:\n")

# Patient without disability
qalys_nondisabled = 10 * (1 - 0)  # 10 years at full health
print(f"Patient A (no disability):")
print(f"  10 years × 1.0 utility = {qalys_nondisabled:.1f} QALYs gained\n")

# Patient with pre-existing paraplegia
qalys_disabled = 10 * (1 - 0.133)  # 10 years with paraplegia DW
print(f"Patient B (pre-existing paraplegia, DW=0.133):")
print(f"  10 years × 0.867 utility = {qalys_disabled:.1f} QALYs gained\n")

print("-"*60)
print(f"If we allocate based on QALYs gained, Patient A's life")
print(f"is valued {qalys_nondisabled/qalys_disabled:.1%} as much as Patient B's.")
print(f"\nIs this fair? This is a genuine ethical debate.")

## 8. Summary and Key Takeaways

### What You've Learned

1. **DALYs combine mortality (YLL) and morbidity (YLD)** into a single measure of disease burden

2. **Disability weights are derived from population surveys** but embed significant value judgements

3. **Your own disability weights likely differ from GBD** — and that's expected. There's no objectively "correct" weight.

4. **Different weights change burden rankings** — and therefore policy priorities

5. **DALYs and QALYs serve different purposes**: burden estimation vs. cost-effectiveness

6. **These measures have limitations** — particularly around equity and disability rights

### For Next Week

In Week 3, we'll apply similar quantitative thinking to **health inequalities** — measuring the gradient between socioeconomic groups and asking how dietary risks contribute to these inequalities.

---

## Exercises

### Exercise 1: Calculate DALYs for a Nutrition-Related Condition

Iron deficiency anaemia is one of the leading causes of YLD globally. Using the data below, calculate the total DALYs.

Data (hypothetical UK population):
- Prevalence of mild anaemia: 800,000
- Prevalence of moderate anaemia: 150,000  
- Prevalence of severe anaemia: 20,000
- Deaths (mostly in elderly with comorbidities): 500 at age 75, 1000 at age 85
- Disability weights: mild=0.004, moderate=0.052, severe=0.149

In [ ]:
# YOUR CODE HERE



### Exercise 2: Sensitivity Analysis

How much would the disability weight for obesity need to increase for it to move from its current ranking to the top 3 conditions in our burden table?

In [ ]:
# YOUR CODE HERE
# Hint: Try different disability weights and recalculate rankings



### Exercise 3: Reflection Questions (No Code Required)

1. The GBD disability weight for obesity (0.086) is lower than for moderate depression (0.145). Do you think this reflects the true burden of these conditions? What factors might the surveys miss?

2. If you were advising Public Health England on measuring the burden of poor diet, would you recommend using DALYs from dietary risk factors (as GBD does) or measuring diet-related conditions separately? What are the trade-offs?

3. The GBD disability weight for Type 2 diabetes without complications is very low (0.015). Yet diabetes is often described as a major public health crisis. Reconcile these two perspectives.

---

## References

- GBD 2019 Diseases and Injuries Collaborators. (2020). Global burden of 369 diseases and injuries in 204 countries and territories. *The Lancet*.
- Salomon JA et al. (2015). Disability weights for the Global Burden of Disease 2013 study. *The Lancet Global Health*.
- Nord E. (2015). Limitations of the QALY − A critical review of the QALY model. *Cambridge Quarterly of Healthcare Ethics*.
- Whitehead SJ, Ali S. (2010). Health outcomes in economic evaluation: the QALY and utilities. *British Medical Bulletin*.